# Dependence Analysis

当编译器要针对并行体系结构进行优化时, 要求编译器可以寻找到有效的数据分解策略, 从而充分利用硬件资源. 需要找到正确的数据分解策略就需要具备判断不同数组索引是否相等的能力,并且可以判断不同的迭代之间是否含有数据依赖, 依赖分析则是其中十分重要的一项技术.

⚠️ 依赖分析过于复杂, 因此本章内容可能不够深入, 建议与参考资料一起看.

参考资料:
1. [零基础入门依赖环的检测](https://zhuanlan.zhihu.com/p/649953141)
2. 《多面体编译理论与深度学习实践》第三章.
3. [polycomp-tutorial 第六章](https://joelburget.com/polycomp-tutorial-v0.02.pdf)

所有教程源码位于[这里](https://github.com/zhen8838/isl_learn).

### Access Relation

访问关系映射实例元素设置为该语句访问的数据元素. 在读取和写入访问之间做出区分通常很重要

- Read Access Relation

  读取访问关系将每个动态执行实例映射到通过动态执行实例读取的数据元素集.

- Write Access Relation

  写入访问关系将每个动态执行实例映射到动态执行实例写入的数据元素集

在某些情况下,无法确定访问的数据元素的确切集. 或者即使有可能确定确切的访问关系,也可能无法将它们表示为`presburger relation`. 因此需要近似访问关系. 如果读取, 则足以确定访问已访问的数据元素的高估集合. 写入的情况需要更多考虑写入访问关系的某些用途, 例如计算程序片段可能访问的元素集, 这允许高估. 而写入访问关系的其他一些用途不允许高估,而是需要低估. 因此出现以下三种类型的访问关系:

- May-Read Access Relation

  包含了读取访问关系作为子集.


- May-Write Access Relation

  包含了写访问关系作为子集.

- Must-Write Access Relation

  是一个写访问关系的子集.

在编译时无法确切计算的访问关系的情况下, 这种灵活性是有用的, 如果给定的动态执行实例将访问哪些元素, 或者如果无法准确表示此信息. 在此信息可用并且可以准确表示的情况下, 可以将访问关系限制/扩展为包括已访问的数据元素, 比如`may-write`关系可以等同于`must-write`关系. 这三个关系都可以以`presburger relation`的方式表示, 同时对于标量的索引用空`tuple`来表示. 

In [62]:
from common import isl, pet, CSource, CodeGenerator, Callable, parse_code

In [63]:
scop = parse_code("""
float inner(int n, float A[const restrict static n], float B[const restrict static n]) {
#pragma scop  
   float prod;
S: prod = 0.0;
L: for (int i = 0; i < n; ++i) 
T:  prod += A[i] * B[i];
#pragma endscop
 return prod; 
}
""", "inner")

我们解析函数后获取三种访问关系:

In [64]:
scop.get_may_reads()

isl.union_map("[n] -> { T[i] -> prod[] : 0 <= i < n; T[i] -> A[i] : 0 <= i < n; T[i] -> B[i] : 0 <= i < n }")

In [65]:
scop.get_may_writes()

isl.union_map("[n] -> { S[] -> prod[]; T[i] -> prod[] : 0 <= i < n }")

In [66]:
scop.get_must_writes()

isl.union_map("[n] -> { S[] -> prod[]; T[i] -> prod[] : 0 <= i < n }")

### Tagged Access Relations

标准访问关系是通过该语句实例访问的数据元素的映射的. 但是给定的`statement`可以多次引用相同的数据结构, 在某些情况下需要区分各个引用. 例如, 当`PPCG`确定要`from/to`设备复制的数据时, 它会检查哪些写入引用产生仅在给定内核内使用的数据. 这要求引用可以从依赖关系中识别, 同时需要它们在访问关系中添加进行不同的标记来区分.

在`pet`中, 为程序片段中的每个引用生成了唯一的标识符. 然后, 这些标识符用于“标记”语句实例, 在所谓的标记访问关系中执行访问. 特别是, 这种标记的访问关系的域是与语句实例和范围作为引用标识符的包装关系. 可以通过计算域产品域因子来从这种标记的访问关系中删除标签. 标记的访问关系可以使用以下功能从`scop`中获取:

In [67]:
scop.get_tagged_may_reads()

isl.union_map("[n] -> { [T[i] -> __pet_ref_4[]] -> B[i] : 0 <= i < n; [T[i] -> __pet_ref_3[]] -> A[i] : 0 <= i < n; [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n }")

In [68]:
scop.get_tagged_may_writes()

isl.union_map("[n] -> { [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n; [S[] -> __pet_ref_1[]] -> prod[] }")

In [69]:
scop.get_tagged_must_writes()

isl.union_map("[n] -> { [T[i] -> __pet_ref_2[]] -> prod[] : 0 <= i < n; [S[] -> __pet_ref_1[]] -> prod[] }")

## Dependence Relation

依赖关系是实例集元素之间的二元关系, 其中一个实例以某种方式依赖于另一个实例. 可以考虑几种类型的依赖关系, 一个实例对另一个实例的依赖性的确切性质取决于依赖关系的类型. 但是通常依赖关系表示一个实例需要在另一个实例之前执行. 依赖性的原因通常是两个语句实例涉及访问相同的内存元素. 可以根据所涉及的两个访问的类型来区分不同类型的依赖.

🟢 Read-after-Write Dependence Relation
  
  如果实例j在实例i之后执行, 并且读取被实例i写过的数据点, 那么写后读依赖实例i映射实例j.

🟢 Write-after-Read Dependence Relatio

  如果实例j在实例i之后执行, 并且写入被实例i读过的数据点, 那么读后写依赖实例i映射实例j.

🟢 Write-after-Write Dependence Relation

  如果实例j在实例i之后执行, 并且写入被实例i写入的数据点, 那么写后写依赖实例i映射实例j.


对于这些依赖关系, 同样可有可能无法被精确计算或者被`presburger relation`所表示. 但是如果只考虑表示执行顺序的先后时, 这些关系可以看作是被高估的. 在后续的教程总所有提到的`Read-after-Write`依赖, 都是`May Read-after-Write`依赖, 其他两个也同样.

`Read-after-Write`依赖是有用的, 因为读取访问可能会读取由写入访问所写的值. 读后写依赖也叫作`anti-dependences`性. 它也是有用的, 因为写入访问可能会覆盖读取访问所读取的值. 写后写依赖也叫作`output dependences`, 也是有用的, 因为第二个写入访问可能会覆盖第一个写入访问所写的值, 在第二次之前执行要执行的第一次写入来确保将最终值写给原始程序中的数据元素的最终值不会出现错误. 而一对读取访问不会产生实际依赖, 因为这两个读取不会相互影响. 但是如果考虑为优化目的读取相同内存元素的一对语句实例, 仍然是有用的. 与真依赖性类似, 此类语句实例有时被称为读后读依赖或`input dependences`, 它与实际依赖的情况相反, 对于输入依赖, 两个语句实例的顺序并不重要.

接下来看一下例子:
```c
for (int i = 0; i < n; ++i) { 
  S: t = A[i]; 
  T: B[i] = t; 
}
```

statement`S`和`T`分别写入变量`t`和读取`t`. 


| |||
|-|-|-|
| 写后读依赖 |$${ S[i] \rightarrow T[i'] : i' \geq i }$$| ![](images/dep_a.png) |
| 读后写依赖 |$${ T[i] \rightarrow S[i'] : i' > i }$$| ![](images/dep_c.png) |
| 写后写依赖 |$${ S[i] \rightarrow S[i'] : i' > i }$$| ![](images/dep_d.png) |

到目前为止所描述的依赖的主要目的是确保在读取值之前将值写入内存, 并且之间没有被覆盖. 在某些情况下, 可能有很多这些依赖性, 以至于实例的执行顺序几乎无法更改甚至根本无法更改. 例如, 如果使用临时标量变量来存储不同的数据, 则上述依赖性将序列化访问该标量变量的语句实例. 通过将不同的数据存储在不同的内存位置中, 其中一些依赖性不再需要, 并且得到更大的自由度来更改语句实例的执行顺序. 有时一些写后读依赖和写后写依赖可以消除, 因此它们也被共同被称为`false dependences`


🟢 Dataflow Dependence Relation

  数据流依赖关系是(确切的)写后读依赖关系的子集, 包含没有对相同的数据进行写入的同一个语句实例.


数据流依赖性也称为`value-based dependences`, 因为该值沿依赖关系保留. 相反, 先前描述的依赖性也称为`memory-based dependences`的依赖性, 因为它们仅访问相同的内存位置.

和之前一样, 也可能无法确定或表示数据流依赖关系, 并且与写入访问一样, 重要的是要区分潜在的数据流和确定的数据流. 所以也有以下两种类型的数据流依赖关系:

- May-Dataflow Dependence Relation

  包含数据流依赖关系作为子集的二元关系.

- Must-Dataflow Dependence Relation

  数据流依赖关系的二元关系子集.

两者都以“标记”形式出现,每个语句实例都伴随着引用标识符,例如之前提到的`Tagged Access Relations`. 这些称为`tagged May-DataFlow`依赖关系和`tagged Must-Dataflow`依赖关系. 通常`Must-Dataflow`依赖关系是`May-Dataflow`依赖关系的子集. 如果可以准确执行数据流分析, 则两者彼此相等. `May-DataFlow`依赖关系本身就是(`May`)`Read-after-Write`依赖关系的子关系. 无标记的`Must-Dataflow`依赖关系仅在每个语句最多包含一个写入访问时才有用.


考虑之前的例子, 通过`S`的实例写入标量`t`的每个值都被同一语句的下一个实例所覆盖. 这意味着该值仅由语句`T`的一个中间实例读取. 也就是说，数据流依赖关系为:

| |||
|-|-|-|
| 数据流依赖 | $$ { S[i] \rightarrow T[i] } $$ | ![](images/dep_b.png) |

## Dependence Analysis

现在开始考虑依赖分析, 从访问关系和调度来计算依赖关系非常容易. 考虑计算写后读依赖关系的例子, 设$W$为`May-Write`访问关系, 设$R$为`May-Read`访问关系, 首先计算对于相同的元素一个执行写入, 一个执行读取的实例对, 可以通过将`May-Write`访问关系与`May-Read`访问关系的反向组合来获得此关系:

$$ R ^{-1} \circ W$$

然后, 保留第一个实例在第二个实例执行之前执行的实例对, 也就是说, 保留下来满足`schedule order`的实例:

$$(R ^{-1} \circ W)\ \cap < S$$

其他的读后写和写后写关系也可以使用类似的方法计算:

⚠️ 这个例子为了简化起见, 同一个实例中没有读后写的依赖, 否则这里的`schedule order`需要修改为`smaller than or equal`.

$$(W ^{-1} \circ R)\ \cap < S$$
$$(W ^{-1} \circ W)\ \cap < S$$


对依赖关系的上述表达的直接评估需要从调度S计算执行顺序关系$< S$, 由于它是在此调度中的成对域元素之间的关系, 相应的范围的元素按词典序排列. 那么, 执行命令关系就是与调度和自身之间词典序小于关系(因为这里例子):
$$< S = \bf{S} ≺ \bf{S} $$


⚠️ : 这里$ B \circ A = { i \rightarrow j : ∃ k : i \rightarrow k \in A \land k \rightarrow j \in B }$, 在isl中为`A.apply_range(B)`.


In [70]:
scop = parse_code("""
float f1(float ); 
float f2(float ); 
void f(int n, float A[ restrict static n], float B[ restrict static n]) { 
#pragma scop  

  float t; 
  for (int i = 0; i < n; ++i) { 
S: t = f1(A[i]); 
T: B[i] = f2(t); 
  }
#pragma endscop   
}
""",'f')

In [71]:
sch =  scop.get_schedule()
root = sch.get_root()
order = sch.get_map() # get map用于获得schedule tree的lex order.
print("order",order)
order_relation = order.lex_lt_union_map(order)
print("order_relation", order_relation)
print("may_writes", scop.get_may_writes())
print("may_reads", scop.get_may_reads())
print("写与读进行composition", scop.get_may_writes().apply_range(scop.get_may_reads().reverse()))
print("写后读依赖", scop.get_may_writes().apply_range(scop.get_may_reads().reverse()).intersect(order_relation))

order [n] -> { T[i] -> [1, i, 1]; S_0[] -> [0, 0, 0]; S[i] -> [1, i, 0] }
order_relation [n] -> { S[i] -> T[i'] : i' > i; S[i] -> T[i' = i]; S[i] -> S[i'] : i' > i; S_0[] -> T[i]; S_0[] -> S[i]; T[i] -> T[i'] : i' > i; T[i] -> S[i'] : i' > i }
may_writes [n] -> { T[i] -> B[i] : 0 <= i < n; S[i] -> t[] : 0 <= i < n }
may_reads [n] -> { T[i] -> t[] : 0 <= i < n; S[i] -> A[i] : 0 <= i < n }
写与读进行composition [n] -> { S[i] -> T[i'] : 0 <= i < n and 0 <= i' < n }
写后读依赖 [n] -> { S[i] -> T[i'] : 0 <= i < n and i' > i and 0 <= i' < n; S[i] -> T[i' = i] : 0 <= i < n }


解释一下计算写后读依赖的过程, 首先`may_writes [n] -> { T[i] -> B[i] : 0 <= i < n; S[i] -> t[] : 0 <= i < n }`表示每个循环中`S`写入标量`t`, `T`写入数组`B`;`may_reads [n] -> { T[i] -> t[] : 0 <= i < n; S[i] -> A[i] : 0 <= i < n }`表示每个循环中`T`读取标量`t`, `S`读取数组`A`. 
将写与读的逆进行`compostion`之后, 也就是写出的目标元素和读入的目标元素进行匹配抵消, 获得写入实例和读入实例的读写关系`[n] -> { S[i] -> T[i'] : 0 <= i < n and 0 <= i' < n }`. 获取读写关系之后再消除不符合顺序关系的部分, 就得到了正确的写后读依赖`[n] -> { S[i] -> T[i'] : 0 <= i < n and i' > i and 0 <= i' < n; S[i] -> T[i' = i] : 0 <= i < n }`. 此时由S写入t, 由T读取t, 因此i与i'的范围相同, 且相等. 由S写入t, 下一次循环T读取t, 因此i与i'的范围相同, 且i’在i之后.

In [72]:
print("读后写依赖", scop.get_may_reads().apply_range(scop.get_may_writes().reverse()).intersect(order_relation))
# 由T读取t, 下一个循环由S进行写入, 因此i与i'的范围相同, 且i’在i之后.
print("写后写依赖", scop.get_may_writes().apply_range(scop.get_may_writes().reverse()).intersect(order_relation))
# 由S写入t, 下一个循环再次写入, 因此i与i'的范围相同, 且i’在i之后.

读后写依赖 [n] -> { T[i] -> S[i'] : 0 <= i < n and i' > i and 0 <= i' < n }
写后写依赖 [n] -> { S[i] -> S[i'] : 0 <= i < n and i' > i and 0 <= i' < n }


## Dataflow Analysis

计算数据流依赖会更加复杂. 假设依赖分析已经被精确计算了, 这就表示所有的读写访问也被精确表示了. 接下来描述一些计算数据流依赖的方法以及各个方法的优缺点.

### Method 1

上面说过, 数据流依赖性是一种没有对同一内存位置的中间写入的写后读依赖性. 计算数据流依赖性的一种方法是删除那些存在此类中间写入的写后读依赖性, 称为被中间写入`killed`. 为了能够将中间写入与正确的写后读依赖性相匹配, 依赖性需要跟踪所涉及的内存元素. 
因此读取访问关系$R$, 取得当前语句读取数据的关系, 在isl中表示为`range map`:

$$ R_1 = \xrightarrow[]{ran}\ R$$

同样, 获取写入访问关系$W$的写入数据关系:

$$ W_1 = \xrightarrow[]{ran} W$$

现在得到了写入和读取对于数据的关系, 然后需要将这些数据关系进行`order`, 所以需要合并读写关系并取`domain`, 再通过`composition`原始`schedule`获得当前访问数据关系`domain`的`order`:

$$ S_1 = S \circ (\xrightarrow[]{dom} (R \cup W)) $$

然后, 与相关数组元素的写后读依赖性参考上面的方法, 可以计算为:

$$ D_1 = (R^{-1}_1 \circ W_1) \cup <S_1 $$

类似的, 对于数组元素的写后写依赖性可以计算为:

$$ O_1 = (W^{-1}_1 \circ W_1) \cup <S_1 $$

然后, 数据流依赖性是写后读依赖, 除去写后写依赖性和写后读依赖性组合部分, 即:

$$ F_1 = D_1 \setminus (D_1 \circ O_1) $$

对数组元素的引用可以通过从$F_1$中删除domain factor来获得.


In [73]:
r1 = scop.get_may_reads().range_map()
print("r1:", r1)
w1 = scop.get_may_writes().range_map()
print("w1:", w1)
S = scop.get_schedule().get_map()
print("S:", S)
s1 = scop.get_may_reads().union(scop.get_may_writes()).domain_map().apply_range(S)
print("s1:", s1)
order_s1 = s1.lex_lt_union_map(s1)
print("order_s1:", order_s1)
d1 = w1.apply_range(r1.reverse()).intersect(order_s1)
print("写后读依赖:", d1)
o1 = w1.apply_range(w1.reverse()).intersect(order_s1)
print("写后写依赖:", o1)
f1 = d1.subtract(o1.apply_range(d1))
print("数据流依赖:", f1)

r1: [n] -> { [T[i] -> t[]] -> t[] : 0 <= i < n; [S[i] -> A[i]] -> A[i] : 0 <= i < n }
w1: [n] -> { [S[i] -> t[]] -> t[] : 0 <= i < n; [T[i] -> B[i]] -> B[i] : 0 <= i < n }
S: [n] -> { T[i] -> [1, i, 1]; S_0[] -> [0, 0, 0]; S[i] -> [1, i, 0] }
s1: [n] -> { [S[i] -> t[]] -> [1, i, 0] : 0 <= i < n; [T[i] -> t[]] -> [1, i, 1] : 0 <= i < n; [T[i] -> B[i]] -> [1, i, 1] : 0 <= i < n; [S[i] -> A[i]] -> [1, i, 0] : 0 <= i < n }
order_s1: [n] -> { [S[i] -> t[]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> B[i]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> B[i]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> t[]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> A[i]] -> [S[i'] -> A[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [T[i] -> t[]] -> [T[i'] -> B[i']] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> t[]] -> [T[i'] -> t[]] : 0 <= i < n and i' > i and 0 <= i' < n; [S[i] -> t[]] -> [T[i' = i] -> t[]] 

上面这种计算数据流依赖性的方法的主要问题是它不能轻松地处理近似值, 因为`D1` 用于中减法运算的两侧, 如果`D1`是高估的, 则不能保证结果是高估还是低估的, 而大多数实际用途需要考虑这个问题.

### Method 2

计算数据流依赖性的另一种方法是将每个对同一内存元素的读取访问与其之前的最后一个写入进行配对. 为了简单起见, 这里假定每个语句实例最多都在一个数据元素上读取或写入, 否则还需要跟踪不同的数据元素. 

这个方法的第一步首先找到读与写的映射, 然后取与执行顺序的反向相交的集合, 即是获取到读前写的关系: 

$$ A  = (W^{-1} \circ R) \cap (<S)^{-1}$$

现在需要从中获得读取前最后一次写入的子集, 首先通过`schedule`将写入实例映射为顺序关系, 然后将写入实例映射到`schedule`中的选择最大的位置的那个, 再将位置映射到写入实例中. 得到读取实例映射最后一次写入实例的关系, 最后数据流依赖关系则是这个关系的逆反:

$$ F = (S^{-1} \circ \text{lexmax}(S \circ A))^{-1} $$

请注意, 该计算假设按`schedule`定义的顺序是全部的, 即`schedule`对每个语句实例分配了不同的位置, 以便可以从该位置恢复语句实例.

In [74]:
S = scop.get_schedule().get_map()
order = S.lex_lt_union_map(S)
A = scop.get_may_reads().apply_range(scop.get_may_writes().reverse()).intersect(order.reverse())
print("读前写关系:" , A)

WriteOrder = A.apply_range(S)
LastWriteOrder = WriteOrder.lexmax()
print("写入的词典序:",WriteOrder)
print("最后一次写入的词典序:",LastWriteOrder)
LastWrite = LastWriteOrder.apply_range(S.reverse())
print("最后一次写入:",LastWrite)
F = LastWrite.reverse()
print("数据流依赖:", F)

读前写关系: [n] -> { T[i] -> S[i'] : 0 <= i < n and 0 <= i' < i and i' < n; T[i] -> S[i' = i] : 0 <= i < n }
写入的词典序: [n] -> { T[i] -> [1, o1, 0] : i < n and 0 <= o1 < i; T[i] -> [1, i, 0] : 0 <= i < n }
最后一次写入的词典序: [n] -> { T[i] -> [1, i, 0] : 0 <= i < n }
最后一次写入: [n] -> { T[i] -> S[i' = i] : 0 <= i < n }
数据流依赖: [n] -> { S[i] -> T[i' = i] : 0 <= i < n }


## Approximate Dataflow Analysis

上面描述了如何执行(简化的)数据流分析. 但是访问关系可能并不总是确切地知道,因为访问可能取决于运行时信息,或者它们可能无法表示为`Presburger relations`. 在这种情况下,数据流分析只能近似计算. 通常有两种计算近似数据流的方法:

1. 操作`may`和`must`的访问关系, 并直接计算近似数据流.

2. 跟踪其他运行时信息, 并得出确切但与运行时有关的数据流依赖关系.

### Compute Approximate Dataflow

之前计算数据流分析的时, 基于内存的依赖分析和基于值的依赖分析之间的主要区别在于, 在基于值的依赖分析中, 是当前写入`kill`在他之前的写入之后中间的读取依赖. 然后, 本节所述的近似数据流分析背后的主要思想是仅允许`must-write`可以`kill`任何依赖. 在最坏的情况下, 所有`may-write`都不是`must-write`, 然后依赖分析的结果与基于内存的依赖分析的结果相同. 为了不过于依赖标准的数据流分析,本操作采用`may-source`、`must-source`和`sink`的形式表达, 而不是`may-write`、`must-write`和`read`的形式.


近似数据流分析使用三个二元关系和一个二元关系`domain`上的`schedule`作为输入, 三个二元关系分别为`sink` $K$,`may-source` $Y$和 `must-source` $T$. `schedule` $S$用于计算`last`、`before`和`after`的位置. 对于`sink`的每个`domain`元素$i$和每个相应的`range`元素$a$, 近似数据流分析确定在$i$之前执行的`must source`的最后一个`domain`元素$j$,并且其相应的`range`元素也是$a$. (也就是与之前的方法2一样, 通过`schedule order`来确定`read`的实例$i$之前执行最后一个`must write`实例$j$, 并且这两个实例访问的是相同的数组元素$a$).

进一步, 近似数据流分析还收集了在$i$之前和$j$之后执行的`may source`的所有`domain`元素$k$, 并且$k$相应的`range`元素也是$a$. 如果在$i$和$a$的待定组合找不到这样的$j$, 则删除`after`$j$的条件. 换句话说, 对于`sink`的每个`domain`元素和每个相应的`range`元素, 之前执行的`must-source`和`may-source`的`domain`元素如果共享这个`range`元素,就会被收集起来,直到找到一个`must source`的`domain`元素. 

最后收集到如下的类似于$j \rightarrow (i \rightarrow a)$ 和 $k \rightarrow (i \rightarrow a)$的元组形成了`may-dependence`关系, 其中没有中间$k$的$j \rightarrow (i \rightarrow a)$子集形成了`must-dependence`关系. 对于找不到对应`domain`元素$j$的子集构成了`may-no-source`关系, 对于既找不到对应`domain`元素$j$也找不到对应`domain`元素$k$的子集构成了`must-no-source`关系.

其中`may-dependence`关系表示如下:

$$
\begin{gathered}
\left\{\boldsymbol{k} \rightarrow(\boldsymbol{i} \rightarrow \boldsymbol{a}): \boldsymbol{i} \rightarrow \boldsymbol{a} \in K \wedge \boldsymbol{k} \rightarrow \boldsymbol{a} \in(T \cup Y) \wedge \boldsymbol{k}<_S \boldsymbol{i} \wedge\right. \\
\left.\neg\left(\exists \boldsymbol{j}: \boldsymbol{j} \rightarrow \boldsymbol{a} \in T \wedge \boldsymbol{k}<_S \boldsymbol{j}<_S \boldsymbol{i}\right)\right\},
\end{gathered}
$$

`must-dependence`关系如下:

$$
\begin{aligned}
\{\boldsymbol{k} \rightarrow(\boldsymbol{i} \rightarrow \boldsymbol{a}): & \boldsymbol{i} \rightarrow \boldsymbol{a} \in K \wedge \boldsymbol{k} \rightarrow \boldsymbol{a} \in T \wedge \boldsymbol{k}<_S \boldsymbol{i} \wedge \\
& \left.\neg\left(\exists \boldsymbol{j}: \boldsymbol{j} \rightarrow \boldsymbol{a} \in(T \cup Y) \wedge \boldsymbol{k}<_S \boldsymbol{j}<_S \boldsymbol{i}\right)\right\},
\end{aligned}
$$

`may-no-source`关系如下:

$$
\left\{\boldsymbol{i} \rightarrow \boldsymbol{a} \in K: \neg\left(\exists \boldsymbol{j}: \boldsymbol{j} \rightarrow \boldsymbol{a} \in T \wedge \boldsymbol{j}<_S \boldsymbol{i}\right)\right\}
$$

`must-no-source`关系如下:

$$
\left\{\boldsymbol{i} \rightarrow \boldsymbol{a} \in K: \neg\left(\exists \boldsymbol{j}: \boldsymbol{j} \rightarrow \boldsymbol{a} \in(T \cup Y) \wedge \boldsymbol{j}<_S \boldsymbol{i}\right)\right\}
$$


在`isl`中, 可通过`isl_union_access_info_compute_flow`函数进行近似数据流分析, 此函数将`isl_union_access_info`对象(描述`may-source`、`must-source`和`sink`和`schedule`)作为输入，并且会产生一个`isl_union_flow`对象(描述了`may-dependence`依赖关系, `may-dependence`依赖性关系, `may no source`关系和`must no source`关系)作为输出. 该实现结合了前面部分中描述的技术，但它避免了构建调度关系$\lt S$. `isl_union_access_info`对象是通过对函数进行调用`isl_union_access_info_from_sink`来构建的. 他的输入可以通过以下函数获得.

- isl_union_access_info_set_must_source 
- isl_union_access_info_set_may_source 
- isl_union_access_info_set_schedule, schedule tree的形式表示时.
- isl_union_access_info_set_schedule_map, Presburger relation的形式表示时.

In [75]:
schedule = scop.get_schedule()
may_read = scop.get_may_reads()
may_write = scop.get_may_writes()
must_write = scop.get_must_writes()
access = isl.union_access_info(may_read)
access = access.set_may_source(may_write)
access = access.set_must_source(must_write)
access = access.set_schedule(schedule)
flow = access.compute_flow()
print("May-flow dependences :", flow.get_may_dependence())
print("May-live-in accesses :", flow.get_may_no_source())


May-flow dependences : [n] -> { S[i] -> T[i' = i] : 0 <= i < n }
May-live-in accesses : [n] -> { S[i] -> A[i] : 0 <= i < n }


In [76]:
schedule = scop.get_schedule()
may_read = scop. get_tagged_may_reads()
may_write = scop. get_tagged_may_writes()
must_write = scop. get_tagged_must_writes()
tagged_instances = may_write .union(may_read). domain()
tagged_instances = tagged_instances .unwrap()
drop_tags = tagged_instances . domain_map_union_pw_multi_aff()
schedule = schedule . pullback(drop_tags)
access = isl. union_access_info(may_read)
access = access. set_may_source(may_write)
access = access. set_must_source(must_write)
access = access. set_schedule(schedule)
flow = access. compute_flow()
print("Tagged may-read access relation :")
print(may_read)
print("Tagged may-write access relation :")
print(may_write)
print("Tagged may-flow dependences :")
print(flow. get_may_dependence())
print("Tagged may-live-in accesses :")
print(flow. get_may_no_source())


Tagged may-read access relation :
[n] -> { [S[i] -> __pet_ref_2[]] -> A[i] : 0 <= i < n; [T[i] -> __pet_ref_4[]] -> t[] : 0 <= i < n }
Tagged may-write access relation :
[n] -> { [S[i] -> __pet_ref_1[]] -> t[] : 0 <= i < n; [T[i] -> __pet_ref_3[]] -> B[i] : 0 <= i < n }
Tagged may-flow dependences :
[n] -> { [S[i] -> __pet_ref_1[]] -> [T[i' = i] -> __pet_ref_4[]] : 0 <= i < n }
Tagged may-live-in accesses :
[n] -> { [S[i] -> __pet_ref_2[]] -> A[i] : 0 <= i < n }


### Kills

如果有许多`may-writes`不是`must-writes`,则近似数据流分析可能非常不准确. 在某些情况下,用户可能能判断通过程序文本中的某个内存片段在某个点上不会发生数据流依赖. 用户可以通过在程序文本中的那个点插入一个 `__pencil_kill` 语句来传达这一信息,杀死给定的内存片段. 具体来说,一个 `__pencil_kill` 语句由对 `__pencil_kill` 的“调用”组成,其参数是在那个点无法流经的数据元素. 这些杀死语句可以然后用于通过将它们包含在近似数据流分析的必须源中来杀死依赖性. 输出可能然后包含从这些杀死语句出现的“依赖性”,但是可以简单地将其删除. 
这些杀死语句被 `pet` 收集并可以通过调用 `pet_scop_get_must_kills` 或 `pet_scop_get_tagged_must_kills` 函数来提取. 
注意,`pet` 为 `__pencil_kill` 调用的每个参数引入一个单独的杀死语句. 所以程序语句上的标签可能不会保留.



In [77]:
scop = parse_code("""
int f(int );
void g(int N, int perm[ restrict static N], int A[ restrict static N]) {
#pragma scop
S: A[0] = 1; 
K: __pencil_kill (A); 
  for (int i = 0; i < N; ++i) 
T:   A[perm[i]] = f(i); 
U: A[0] = f(A[0]); 
#pragma endscop
}""", 'g')

schedule = scop. get_schedule()
may_read = scop. get_may_reads()
may_write = scop. get_may_writes()
must_write = scop. get_must_writes()
kill = scop. get_must_kills()
access = isl. union_access_info(may_read)
access = access. set_may_source(may_write)
access = access. set_must_source(must_write)
access = access. set_schedule(schedule)
flow1 = access. compute_flow()
access = access. set_must_source(must_write .union(kill))
flow2 = access. compute_flow()
print("May-flow dependences without kills:")
print(flow1. get_may_dependence())
print("Kills:")
print(kill)
print("May-flow dependences with kills:")
f = flow2. get_may_dependence()
f = f. subtract_domain(kill.domain())
print(f)


May-flow dependences without kills:
[N] -> { T[i] -> U[] : 0 <= i < N; S[] -> U[] : N > 0 }
Kills:
[N] -> { S_1[] -> A[o0] : 0 <= o0 < N }
May-flow dependences with kills:
[N] -> { T[i] -> U[] : 0 <= i < N }


上面的代码中, 假设`perm`是`0`到`N-1`的整数的排列. 这意味着由于`A[0]`被语句`T`的某个实例覆盖,所以在语句`S`和语句`U`之间不会发生数据流依赖. 然而,编译器不知道`perm`是一个排列. 因此,添加了一个调用`__pencil_kill`的语句,表示整个数组`A`由`T`的实例`kill`. 下面的记录显示了在数据流分析期间考虑杀死语句的效果. 如果不添加`kill`,那么会存在一个额外的从语句`S`到语句`U`的依赖性. 


### Live-out Accesses

近似数据流分析也可以用于计算`may-live-out`的关系. 这是一个包含可能仍然有相应的读操作在程序片段分析之外的 `may-write` 访问的关系. 特别是, 它包含所有的 `may-write` 访问,这些访问没有被后来的必须写入`must-write`访问或`kill`消除. 这个被杀死的 `may-write` 访问的集合可以通过将 `must-writes` 和 `kills` 作为 `sinks`,`may-writes` 作为 `may-sources` 来执行数据流分析来计算. 然后可以从所有访问的集合中删除结果 `may-dependence` 关系的定义域. 注意,考虑完整的 `may-dependence` 关系(包括相应的被访问元素)是很重要的,因为对一个 `may-write` 访问的一个元素的 `kill` 不应该被认为是对那个写访问的所有被访问元素的 `kill`.  

In [78]:
scop = parse_code("""
int f(int ); 
void g(int n, int A[const restrict static n]) 
{ 
#pragma scop 
S: A[f(0)] = 1;
T: A[0] = 0;
#pragma endscop 
}
""",'g')
schedule = scop. get_schedule()
may_write = scop. get_may_writes()
must_write = scop. get_must_writes()
kill = scop. get_must_kills()
access = isl. union_access_info(must_write .union(kill))
access = access. set_may_source(may_write)
access = access. set_schedule(schedule)
flow = access. compute_flow()
dep = flow. get_full_may_dependence()
killed = dep. range_factor_range()
live_out = may_write . subtract(killed)
print(live_out)


[n] -> { S[] -> A[o0] : 0 < o0 < n; T[] -> A[0] : n > 0 }
